In [1]:
!pip install numpy matplotlib

# 🌌 Pulsar + Yıldız Tarlası 3D Animasyonu Açıklaması

Bu kod, bir **pulsar**'ın 3D animasyonunu oluşturur. Pulsar, nötron yıldızlarının hızlı dönen ve manyetik kutuplarından ışık (radyo dalgaları) yayan özel türleridir.  
Animasyonda pulsarın dönüşü, ışık konileri ve arkada yıldız tarlası birlikte gösterilir.

---

## 1️⃣ Kullanılan Kütüphaneler
- **numpy**: Matematiksel hesaplamalar ve veri oluşturma.
- **matplotlib**: 3D çizim ve animasyon oluşturma.
- **IPython.display.HTML**: Colab üzerinde animasyonu HTML5 formatında göstermek için.

---

## 2️⃣ Fiziksel ve Görsel Parametreler
- `rotation_speed`: Pulsarın her karede döneceği derece miktarı (derece/frame).
- `beam_angle`: Işık konisinin (ışın hüzmesinin) açısı (derece).
- `frames`: Toplam kare sayısı (animasyon uzunluğu).

---

## 3️⃣ Pulsar Modeli
- Küre modeli, **küresel koordinatlar** kullanılarak `xs, ys, zs` ile oluşturulur.
- Pulsar turuncu renkte gösterilir ve merkezde konumlandırılır.
- Küre yarıçapı 0.5 birim olarak alınmıştır.

---

## 4️⃣ Işık Konileri (Beam)
- `beam_cone(angle_deg)` fonksiyonu, verilen bir açıyla yukarı ve aşağı yönlü iki koni oluşturur.
- Bu koniler pulsarın manyetik kutuplarından çıkan ışımaları temsil eder.
- Kuzey kutbu: `Xb1, Yb1, Zb1`
- Güney kutbu: `Xb2, Yb2, Zb2` (Z ekseni negatif yönlü).

---

## 5️⃣ Yıldız Tarlası
- `num_stars` kadar rastgele pozisyonda yıldız üretilir (`star_x, star_y, star_z`).
- Her yıldız için `star_brightness` belirlenir, böylece bazı yıldızlar daha parlak görünür.
- Arka planda 3D uzay hissi vermek için geniş bir alana dağıtılır.

---

## 6️⃣ Animasyon Mantığı (`update(frame)`)
- **Arka plan yıldızları** önce çizilir (`ax.scatter`).
- **Pulsar** küresi ortada çizilir (`ax.plot_surface`).
- **Dönme hareketi**: `rot` ile her karede belli bir açı döndürülür.
- **Işık konilerinin yönü**: X-Y koordinat dönüşümü ile koniler döndürülür.
- **Parlaklık efekti**: Işık gözlemciye döndüğünde `brightness = cos(rot)` ile parlaklık artırılır.
- **Kamera hareketi**: `ax.view_init` ile kamera açısı hafifçe döndürülür.

---

## 7️⃣ Animasyonu Çalıştırma
- `FuncAnimation` ile animasyon oluşturulur.
- `HTML(anim.to_jshtml())` ile Colab üzerinde HTML5 formatında canlı olarak oynatılır.

---

## 🔍 Fiziksel Gerçekçilik Notu
Bu animasyon görsel amaçlıdır.  
Gerçek pulsarlarda:
- Dönme hızı saniyede yüzlerce devir olabilir.
- Radyo dalgaları görünür ışıkta değil, radyo teleskoplarında tespit edilir.
- Buradaki koniler, görsel temsil için beyaz renkte ve görünür ışık gibi çizilmiştir.

---


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# === Pulsar parametreleri ===
rotation_speed = 5   # derece/frame
beam_angle = 20      # derece (ışının genişliği)
frames = 200

# === 3D figür ===
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')

# Pulsar küresi
u = np.linspace(0, 2*np.pi, 40)
v = np.linspace(0, np.pi, 40)
xs = 0.5 * np.outer(np.cos(u), np.sin(v))
ys = 0.5 * np.outer(np.sin(u), np.sin(v))
zs = 0.5 * np.outer(np.ones_like(u), np.cos(v))

# === Işık huzmesi fonksiyonu ===
def beam_cone(angle_deg):
    angle = np.radians(angle_deg)
    phi = np.linspace(0, 2*np.pi, 50)
    z = np.linspace(0, 3, 50)
    Z, PHI = np.meshgrid(z, phi)
    X = Z * np.tan(angle) * np.cos(PHI)
    Y = Z * np.tan(angle) * np.sin(PHI)
    return X, Y, Z

# Kuzey ve güney kutbundaki ışık konileri
Xb1, Yb1, Zb1 = beam_cone(beam_angle)
Xb2, Yb2, Zb2 = beam_cone(beam_angle)
Zb2 = -Zb2  # aşağı yönlü

# === Yıldız tarlası ===
num_stars = 300
star_x = np.random.uniform(-10, 10, num_stars)
star_y = np.random.uniform(-10, 10, num_stars)
star_z = np.random.uniform(-10, 10, num_stars)
star_brightness = np.random.uniform(0.5, 1.0, num_stars)  # farklı parlaklıklar

# === Animasyon fonksiyonu ===
def update(frame):
    ax.cla()
    
    # Yıldızlar (arka plan)
    ax.scatter(star_x, star_y, star_z, s=2, c=[[1,1,1,b] for b in star_brightness], alpha=0.8)
    
    # Pulsar
    ax.plot_surface(xs, ys, zs, color="orange", alpha=1.0)
    
    # Dönme açısı
    rot = np.radians(frame * rotation_speed)
    cos_r, sin_r = np.cos(rot), np.sin(rot)
    
    # Işınların yönünü döndür
    X1_rot = Xb1*cos_r - Yb1*sin_r
    Y1_rot = Xb1*sin_r + Yb1*cos_r
    X2_rot = Xb2*cos_r - Yb2*sin_r
    Y2_rot = Xb2*sin_r + Yb2*cos_r
    
    # Parlama etkisi
    brightness = max(0, np.cos(rot))
    color_intensity = (1, 1, 1, brightness)  # RGBA
    
    # Işık konileri
    ax.plot_surface(X1_rot, Y1_rot, Zb1, color=color_intensity, alpha=0.7)
    ax.plot_surface(X2_rot, Y2_rot, Zb2, color=color_intensity, alpha=0.7)
    
    # Kamera ayarları
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    ax.set_zlim(-5, 5)
    ax.view_init(elev=20, azim=frame*0.8)  # Hafif dönme
    ax.set_axis_off()

# === Animasyonu oluştur ===
anim = FuncAnimation(fig, update, frames=frames, interval=60, blit=False)
HTML(anim.to_jshtml())
